In [1]:
import numpy as np

## Homework

In this homework, we'll deploy the lead scoring model from the homework 5.

We already have a docker image for this model - we'll use it for 
deploying the model to Kubernetes.


## Building the image

Clone the course repo if you haven't:

```
git clone https://github.com/DataTalksClub/machine-learning-zoomcamp.git
```

Go to the `course-zoomcamp/cohorts/2025/05-deployment/homework` folder and 
execute the following:


```bash
docker build -f Dockerfile_full -t zoomcamp-model:3.13.10-hw10 .
```


## Question 1

Run it to test that it's working locally:

```bash
docker run -it --rm -p 9696:9696 zoomcamp-model:3.13.10-hw10
```

And in another terminal, execute `q6_test.py` file:

```bash
python q6_test.py
```

You should see this:

```python
{'conversion_probability': <value>, 'conversion': False}
```

Here `<value>` is the probability of getting a subscription. You need to choose the right one.

* 0.29
* 0.49
* 0.69
* 0.89

Now you can stop the container running in Docker.


## Installing `kubectl` and `kind`

You need to install:

* `kubectl` - https://kubernetes.io/docs/tasks/tools/ (you might already have it - check before installing)
* `kind` - https://kind.sigs.k8s.io/docs/user/quick-start/


install kind:
[ $(uname -m) = x86_64 ] && curl -Lo ./kind https://kind.sigs.k8s.io/dl/v0.30.0/kind-linux-amd64

chmod +x ./kind

mv ./kind /some-dir-in-your-PATH/kind


## Question 2

What's the version of `kind` that you have? 

Use `kind --version` to find out.

```
./kind --version
kind version 0.30.0
```

## Creating a cluster

Now let's create a cluster with `kind`:

```bash
kind create cluster
```

And check with `kubectl` that it was successfully created:

```bash
kubectl cluster-info

```
kubectl cluster-info
Kubernetes control plane is running at https://127.0.0.1:43271
CoreDNS is running at https://127.0.0.1:43271/api/v1/namespaces/kube-system/services/kube-dns:dns/proxy

To further debug and diagnose cluster problems, use 'kubectl cluster-info dump'.
```

## Question 3

What's the smallest deployable computing unit that we can create and manage 
in Kubernetes (`kind` in our case)?

* Node
* Pod
* Deployment
* Service

Pods

## Question 4

Now let's test if everything works. Use `kubectl` to get the list of running services.

What's the `Type` of the service that is already running there?

* NodePort
* ClusterIP
* ExternalName
* LoadBalancer

```
kubectl get svc -A

NAMESPACE     NAME         TYPE        CLUSTER-IP   EXTERNAL-IP   PORT(S)                  AGE
default       kubernetes   ClusterIP   10.96.0.1    <none>        443/TCP                  7m37s
kube-system   kube-dns     ClusterIP   10.96.0.10   <none>        53/UDP,53/TCP,9153/TCP   7m36s
```

## Question 5

To be able to use the docker image we previously created (`zoomcamp-model:3.13.10-hw10`),
we need to register it with `kind`.

What's the command we need to run for that?

* `kind create cluster`
* `kind build node-image`
* `kind load docker-image`
* `kubectl apply`

kind load docker-image

## Question 6

Now let's create a deployment config (e.g. `deployment.yaml`):

```yaml
apiVersion: apps/v1
kind: Deployment
metadata:
  name: subscription
spec:
  selector:
    matchLabels:
      app: subscription
  replicas: 1
  template:
    metadata:
      labels:
        app: subscription
    spec:
      containers:
      - name: subscription
        image: <Image>
        resources:
          requests:
            memory: "64Mi"
            cpu: "100m"            
          limits:
            memory: <Memory>
            cpu: <CPU>
        ports:
        - containerPort: <Port>
```

Replace `<Image>`, `<Memory>`, `<CPU>`, `<Port>` with the correct values.

What is the value for `<Port>`?

Apply this deployment using the appropriate command and get a list of running Pods. 
You can see one running Pod.

port 9696

load docker image into kubernetes
```
kind load docker-image zoomcamp-model:3.13.10-hw10
```

```
../kind load docker-image zoomcamp-model:3.13.10-hw10
Image: "zoomcamp-model:3.13.10-hw10" with ID "sha256:db7f7db106ba7b284d567329718da64fefba13c61be0574c328fec268bc5b5f7" not yet present on node "kind-control-plane", loading...
```


deploy deployment.yaml to kubernetes

```
kubectl apply -f deployment.yaml 
deployment.apps/subscription created
```

check pods

```
kubectl get pods
NAME                           READY   STATUS    RESTARTS   AGE
subscription-568c5d8cf-b92kd   1/1     Running   0          3s
```


```
kubectl get pods
# delete pod
kubectl delete pod -l app=subscription
# check again
kubectl get pods
```

kubernetes automatically recreate the pod after delete.

to delete permanently;
```
kubectl delete -f deployment.yaml
```



```
kubectl port-forward pod/subscription-568c5d8cf-b92kd 9696:9696
```

## Question 7

Let's create a service for this deployment (`service.yaml`):

```yaml
apiVersion: v1
kind: Service
metadata:
  name: <Service name>
spec:
  type: LoadBalancer
  selector:
    app: <???>
  ports:
  - port: 80
    targetPort: <PORT>
```

Fill it in. What do we need to write instead of `<???>`?

Apply this config file.


## Testing the service

We can test our service locally by forwarding the port 9696 on our computer 
to the port 80 on the service:

```bash
kubectl port-forward service/<Service name> 9696:80
```

Run `q6_test.py` (from the homework 5) once again to verify that everything is working. 
You should get the same result as in Question 1.

```
apiVersion: v1
kind: Service
metadata:
  name: subscription
spec:
  type: LoadBalancer
  selector:
    app: subscription
  ports:
  - port: 80
    targetPort: 9696
```

apply service

```
kubectl apply -f service.yaml

service/subscription created
```

forward port

```
kubectl port-forward service/subscription 9696:80

Forwarding from 127.0.0.1:9696 -> 9696
Forwarding from [::1]:9696 -> 9696
```

```
python q6_test.py 

{'conversion_probability': 0.49999999999842815, 'conversion': False}
```